In [36]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import datasets, linear_model, metrics

pd.set_option('display.float_format',lambda x: '%.4f' %x)
pd.options.mode.chained_assignment = None

train_data = pd.read_csv('data/trainingset.txt',header=None, na_values="?")
test_data = pd.read_csv('data/queries.txt',header=None, na_values="?")
#set column names for train_data
train_data.columns = ['id','age','job','marital','education','default','balance','housing','loan','contact','day','month','duration','campaign','pdays','previous','poutcome','output']
test_data.columns = ['id','age','job','marital','education','default','balance','housing','loan','contact','day','month','duration','campaign','pdays','previous','poutcome','output']

"""
Data Set Information: 

The classification goal is to predict if the client will subscribe a term deposit after they have been contacted by phone (variable output).

Attribute Information:

Input variables:
1 - id
2 - age (numeric)
3 - job (categorical with 12 levels: 'JobCat1' to 'JobCat11', and 'unknown')
4 - marital (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)
5 - education (categorical: "unknown","secondary","primary","tertiary")
6 - default: (bool: "yes","no")
7 - balance: (numeric) 
8 - housing: (bool: "yes","no")
9 - loan:  (bool: "yes","no")
# related with the last contact of the current campaign:
10 - contact: (categorical: "unknown","telephone","cellular") 
11 - day: (numeric)
12 - month: (categorical: "jan", "feb", "mar", ..., "nov", "dec")
13 - duration: (numeric)
# other attributes:
14 - campaign: (numeric, includes last contact)
15 - pdays: (numeric, -1 means client was not previously contacted)
16 - previous: (numeric)
17 - poutcome: (categorical: "unknown","other","failure","success")
Output variable (desired target):
18 - output - has the client subscribed a term deposit? (bool: "TypeA","TypeB")
"""
#assign the datatypes to the columns
train_data.age = train_data.age.astype('int64')
train_data.job = train_data.job.astype('category')
train_data.marital = train_data.marital.astype('category')
train_data.education = train_data.education.astype('category')
train_data.default = train_data.default.map({'yes':1,'no':0})
train_data.balance = train_data.balance.astype('float64')
train_data.housing = train_data.housing.map({'yes': 1, 'no': 0})
train_data.loan = train_data.loan.map({'yes': 1, 'no': 0})
train_data.contact = train_data.contact.astype('category')
train_data.day = train_data.day.astype('int64')
train_data.month = train_data.month.astype('category')
train_data.duration = train_data.duration.astype('int64')
train_data.campaign = train_data.campaign.astype('int64')
train_data.pdays = train_data.pdays.astype('int64')
train_data.previous = train_data.previous.astype('int64')
train_data.poutcome = train_data.poutcome.astype('category')
train_data.output = train_data.output.map({'TypeA': 1, 'TypeB': 0})

#assign the same datatypes to the columns of the test_data
test_data.age = test_data.age.astype('int64')
test_data.job = test_data.job.astype('category')
test_data.marital = test_data.marital.astype('category')
test_data.education = test_data.education.astype('category')
test_data.default = test_data.default.map({'yes':1,'no':0})
test_data.balance = test_data.balance.astype('float64')
test_data.housing = test_data.housing.map({'yes': 1, 'no': 0})
test_data.loan = test_data.loan.map({'yes': 1, 'no': 0})
test_data.contact = test_data.contact.astype('category')
test_data.day = test_data.day.astype('int64')
test_data.month = test_data.month.astype('category')
test_data.duration = test_data.duration.astype('int64')
test_data.campaign = test_data.campaign.astype('int64')
test_data.pdays = test_data.pdays.astype('int64')
test_data.previous = test_data.previous.astype('int64')
test_data.poutcome = test_data.poutcome.astype('category')
test_data.output = test_data.output.map({'TypeA': 1, 'TypeB': 0})

#map the categorical variables to numerical values
train_data.job = train_data.job.cat.codes
train_data.marital = train_data.marital.cat.codes
train_data.education = train_data.education.cat.codes
train_data.contact = train_data.contact.cat.codes
train_data.month = train_data.month.cat.codes
train_data.poutcome = train_data.poutcome.cat.codes

test_data.job = test_data.job.cat.codes
test_data.marital = test_data.marital.cat.codes
test_data.education = test_data.education.cat.codes
test_data.contact = test_data.contact.cat.codes
test_data.month = test_data.month.cat.codes
test_data.poutcome = test_data.poutcome.cat.codes

#create a function that makes a linear regression model for the given data






Coefficients:  [ 3.48917463e-04  3.45018747e-02 -3.69329948e-06  9.63802688e-02
  4.63414041e-02  6.95934192e-04 -1.73472348e-17  6.66863657e-03
 -2.45001953e-04 -1.40582718e-02]


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
train_data.isna().sum()

age         0
default     0
balance     0
housing     0
loan        0
day         0
duration    0
campaign    0
pdays       0
previous    0
output      0
dtype: int64